In [197]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd
from datetime import datetime
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [198]:
#def init_browser():
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
#return Browser("chrome", **executable_path, headless=False)



In [199]:
def init_db():
    # Initialize PyMongo to work with MongoDBs
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.mars_db
    return db


In [200]:
# Define database and collection
#db = client.mars_db
#collection = db.??

In [201]:
def create_soup(url):
    # URL of page to be scraped
   # browser = init_browser()
    browser.visit(url)
    soup = BeautifulSoup(browser.html,"html.parser")

    return soup



In [202]:
def get_mars_news():
    url = 'https://mars.nasa.gov/news/'
    soup = create_soup(url)
    title = soup.find("div",class_ = "content_title").text
    body = soup.find("div", class_="article_teaser_body").text
    return{
    (f"Title: {title}"),
    (f"Para: {body}")
    }
    

In [203]:
def get_mars_imageX():
    #featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    soup = create_soup(url)
    #click full image
    imgElem= soup.find_by_id("full_image")
    imgElem.click()
    soup.is_element_present_by_text("more info",wait_time=0.3)
    moreElem = soup.find_link_by_partial_text("more info")
    moreElem.click()
    imgSoup = create_soup(url)
    imgUrl=imgSoup.select_one("figure.lede a img").get("src")
    imgUrl=f"https://www.jpl.nasa.gov{imgUrl}"
    return {
            "img_url":imgUrl
           }
    print(imgUrl)


In [204]:
def get_MARS_image():
    JPL_home_url = "https://www.jpl.nasa.gov"
    JPL_img_url = JPL_home_url+"/spaceimages/?search=&category=Mars"
    jpl_soup = create_soup(JPL_img_url)
    items = jpl_soup.find("div",{"class":"carousel_items"})
    img_title = items.find("h1",{"class":"media_feature_title"})
    featured_img = items.find("article")
    img_url = JPL_home_url+featured_img['style'].split(':')[1].split('\'')[1]
    return {
            "title":img_title,
            "img_url":img_url
           }

In [205]:
### Mars Weather
#* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.
#python
# Example:
#mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

In [206]:
def get_mars_weather():
    url_weather = "https://twitter.com/marswxreport?lang=en"
    soup = create_soup(url_weather)
    mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
  #  return(mars_weather) 
    print(mars_weather)

In [207]:
#Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
# Use Pandas to convert the data to a HTML table string.
def get_mars_facts():
    url_weather = "https://space-facts.com/mars/"
    table = pd.read_html(url_weather)
    # this is to ensure that we get the first table.
    df = table[0]
    df.columns =["Parameter","Value"]
    df.set_index(["Parameter"])
    table[0]
    new_html_table = df.to_html()
    return(new_html_table)


In [208]:
def get_MARS_hemisphere_data():
    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    soup = create_soup(url)
    items = soup.find("div",{"id":"product-section"}).find_all("div",{"class":"item"})
    hemisphere_data = []
    for item in items:
        print(item)
        img_main_url = "https://astrogeology.usgs.gov"+item.find("a")["href"]
        img_title = item.find("div",{"class":"description"}).find("a").find("h3").text
        img_soup = create_soup(img_main_url)
        download_item = img_soup.find("div",{"class":"downloads"})
        hemisphere_item = {
            "title":img_title,
            "img_url": download_item.find("li").find("a")["href"]
        }
        hemisphere_data.append(hemisphere_item)
        
    return hemisphere_data





In [209]:
  # def getValue():
  #  mars_hm_data = get_MARS_hemisphere_data()
  #  return{
  #        "hemisphere_data":mars_hm_data
  #        }
    


In [210]:
#getValue()

In [211]:
def scrape():
    mars_news = get_mars_news()
    mars_img = get_MARS_image()
    mars_facts = get_mars_facts()
    mars_temp = get_mars_weather()
    mars_hm_data = get_MARS_hemisphere_data()
    #mars_hm_data = getValue()
    return {
        "date":datetime.now(),
        "news":mars_news,
        "featured_img":mars_img,
        "facts":mars_facts,
        "temperature":mars_temp,
        "hemisphere_data":mars_hm_data
    }

In [212]:
scrape()

InSight sol 106 (2019-03-15) low -96.3ºC (-141.4ºF) high -15.9ºC (3.3ºF)
winds from the SW at 4.1 m/s (9.3 mph) gusting to 12.4 m/s (27.8 mph)
pressure at 7.20 hPapic.twitter.com/OkdWbeg6uw
<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>
<div class="item"><a class="itemLink product

{'date': datetime.datetime(2019, 3, 18, 7, 16, 25, 117292),
 'news': {"Para: NASA declared the record-setting rover's mission complete on Feb. 13, 2019, but the final image from the rover has just been processed.",
  "Title: Opportunity's Parting Shot Was a Beautiful Panorama"},
 'featured_img': {'title': <h1 class="media_feature_title">
  				  Weighing in on the Dumbbell Nebula				</h1>,
  'img_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14417-1920x1200.jpg'},
 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Parameter</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n   

In [213]:
df = pd.read_html("https://space-facts.com/mars/")[0]
df = df.rename(columns={0:"Description",1:"Value"})
df = df.set_index("Description")
df.to_dict()['Value']

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-153 to 20 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}